In [ ]:
## Global Requirements :

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from osgeo import gdal
import sys
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data collection :

## 1- Field Measurements :

In [ ]:
## Install Requirements : 

!pip install cartopy

In [ ]:
## Install Requirements : 

pip install ismn

In [ ]:
from ismn.interface import ISMN_Interface

In [ ]:
## Process tha data file with the ISMN interface : 

data_path = "/content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_yyUQ_20210804"
ismn_data = ISMN_Interface(data_path, parallel=True)

Files Processed:   0%|          | 0/1 [00:00<?, ?it/s]

Processing metadata for all ismn stations into folder /content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_yyUQ_20210804.
This may take a few minutes, but is only done once...



Files Processed: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]

Metadata generation finished after 1 Seconds.
Metadata and Log stored in /content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_yyUQ_20210804/python_metadata
Found existing ismn metadata in /content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_yyUQ_20210804/python_metadata/Data_separate_files_header_20170301_20201231_8964_yyUQ_20210804.csv.


In [ ]:
ismn_data.collection

REMEDHUS: ['Llanos_de_la_Boveda']

In [ ]:
 ismn_data.collection['REMEDHUS']['Llanos_de_la_Boveda'].metadata

MetaData([
  MetaVar([clay_fraction, 49.0, Depth([0.0, 0.3])]),
  MetaVar([climate_KG, BSk, None]),
  MetaVar([climate_insitu, unknown, None]),
  MetaVar([elevation, -99.9, None]),
  MetaVar([instrument, Stevens-Hydra-Probe, Depth([0.0, 0.05])]),
  MetaVar([latitude, 41.35757, None]),
  MetaVar([lc_2000, 10, None]),
  MetaVar([lc_2005, 10, None]),
  MetaVar([lc_2010, 10, None]),
  MetaVar([lc_insitu, unknown, None]),
  MetaVar([longitude, -5.33113, None]),
  MetaVar([network, REMEDHUS, None]),
  MetaVar([organic_carbon, 0.87, Depth([0.0, 0.3])]),
  MetaVar([sand_fraction, 19.0, Depth([0.0, 0.3])]),
  MetaVar([saturation, 0.5, Depth([0.0, 0.3])]),
  MetaVar([silt_fraction, 32.0, Depth([0.0, 0.3])]),
  MetaVar([station, Llanos_de_la_Boveda, None]),
  MetaVar([timerange_from, 2017-03-01 00:00:00, None]),
  MetaVar([timerange_to, 2020-12-31 23:00:00, None]),
  MetaVar([variable, soil_temperature, Depth([0.0, 0.05])]),
  MetaVar([variable, soil_moisture, Depth([0.0, 0.05])])
])

In [ ]:
 ismn_data.collection['REMEDHUS']['Llanos_de_la_Boveda']

Sensors at 'Llanos_de_la_Boveda': ['Stevens-Hydra-Probe_soil_temperature_0.000000_0.050000', 'Stevens-Hydra-Probe_soil_moisture_0.000000_0.050000']

In [ ]:
L=['Stevens-Hydra-Probe_soil_moisture_0.000000_0.050000']

In [ ]:
## Utilities :

def get_sensor_by(network, station):
  df = []
  
  for index,value in enumerate(ismn_data.collection[network][station]) :
    #print("soil_moisture" in str(value),index)
    #print("--------------")
    sensor = ismn_data.collection[str(network)][str(station)][str(value)]
    
    if str(value) in L :

      ts = sensor.read_data()
      ts.drop('soil_moisture_orig_flag', inplace=True, axis=1)  
      ts.drop('soil_moisture_flag', inplace=True, axis=1)  
      ts["depth"] = float(str(value).split("_")[-1])
      ts.rename(columns={'soil_moisture': 'soil_moisture_depth_'+str(value).split("_")[-1]},inplace=True)
      df.append(ts)
      allS = pd.concat(df)

  
  return allS

def get_station_by(network):
  df1 = []
  for index,key in enumerate(ismn_data.collection[network]) :
    #print(get_name(key))
    data = get_sensor_by(network, get_name(key))
    data["station"] = get_name(key)
    df1.append(data)

  allN = pd.concat(df1)
  allN.head()
  return allN 



In [ ]:
## Get The dataset of the network-station desired : 

dataset = get_sensor_by('REMEDHUS','Llanos_de_la_Boveda')

In [ ]:
pd.set_option("display.max_rows", False)

In [ ]:
dataset.head()

,soil_moisture_depth_0.050000,depth
date_time,,
2017-03-01 00:00:00,0.238,0.05
2017-03-01 01:00:00,0.238,0.05
2017-03-01 02:00:00,0.237,0.05
2017-03-01 03:00:00,0.236,0.05
2017-03-01 04:00:00,0.236,0.05


In [ ]:
dataset.describe()

,soil_moisture_depth_0.050000,depth
count,33283.000000,3.364800e+04
mean,0.205819,5.000000e-02
std,0.083589,2.919930e-14
min,0.085000,5.000000e-02
25%,0.139000,5.000000e-02
50%,0.205000,5.000000e-02
75%,0.246000,5.000000e-02
max,0.569000,5.000000e-02


In [ ]:
## Save the dataset as csv file :

dataset.to_csv ('SMC_dataset_2017to2020.csv', header=True)

In [ ]:
## Check if the file is correctly saved :

df = pd.read_csv('SMC_dataset_2017to2020.csv')
df.tail()

,date_time,soil_moisture_depth_0.050000,depth
33643,2020-12-31 19:00:00,0.456,0.05
33644,2020-12-31 20:00:00,0.460,0.05
33645,2020-12-31 21:00:00,0.459,0.05
33646,2020-12-31 22:00:00,0.459,0.05
33647,2020-12-31 23:00:00,0.459,0.05


## 2- Satellite Imagery :

In [ ]:
## Requirements : 

import ee
import folium
from IPython.display import Image
from datetime import datetime
from datetime import timedelta

In [ ]:
ee.Authenticate()

In [ ]:
# Initialize the library.
ee.Initialize()

### Selection of a the region

In [ ]:
pip install geemap

In [ ]:
import geemap

In [ ]:
## Downloading images of the region :

lat1 =   -5.3339194363612314
long1 = 41.36287691309333
lat2 =   -5.329676818510049
long2 = 41.353247629449925 
lat3 = -5.34442
long3 = 41.35014 
lat4 =  -5.348314032642031
long4 = 41.35999585947626 


aoi = ee.Geometry.Polygon([[[lat1, long1], [lat2, long2], [lat3, long3], [lat4, long4]]])

CLOUD_FILTER = 60

time_range=['2017-03-29', '2020-12-31']

sentinel_filtered = (ee.ImageCollection('COPERNICUS/S2_SR').
                         filterBounds(aoi).
                         select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
                         )


geemap.ee_export_image_collection_to_drive(sentinel_filtered, folder='Senti_Images-2017to2020_newArea', scale=10, region=aoi)


Total number of images: 484

Exporting 20170412T110621_20170412T111708_T30TUL ...
Exporting 20170502T110621_20170502T110937_T29TQF ...
Exporting 20170502T110621_20170502T110937_T30TTL ...
Exporting 20170502T110621_20170502T110937_T30TUL ...
Exporting 20170512T110651_20170512T111302_T30TTL ...
Exporting 20170522T110621_20170522T110912_T29TQF ...
Exporting 20170522T110621_20170522T110912_T30TTL ...
Exporting 20170522T110621_20170522T110912_T30TUL ...
Exporting 20170601T110651_20170601T111225_T29TQF ...
Exporting 20170601T110651_20170601T111225_T30TTL ...
Exporting 20170601T110651_20170601T111225_T30TUL ...
Exporting 20170611T110621_20170611T111012_T29TQF ...
Exporting 20170611T110621_20170611T111012_T30TTL ...
Exporting 20170611T110621_20170611T111012_T30TUL ...
Exporting 20170621T110651_20170621T111222_T29TQF ...
Exporting 20170621T110651_20170621T111222_T30TTL ...
Exporting 20170621T110651_20170621T111222_T30TUL ...
Exporting 20170701T111051_20170701T111746_T29TQF ...
Exporting 2017070

# 3- Dataset :
Synchronize the field measurement with images dates

In [ ]:
## Requirements :

from osgeo import gdal
import sys
import os

In [ ]:
## Read the data from the 2 stations :

df1 = pd.read_csv("/content/SMC_dataset_2017to2020.csv")

In [ ]:
df1.head() 

,date_time,soil_moisture_depth_0.050000,depth
0,2017-03-01 00:00:00,0.238,0.05
1,2017-03-01 01:00:00,0.238,0.05
2,2017-03-01 02:00:00,0.237,0.05
3,2017-03-01 03:00:00,0.236,0.05
4,2017-03-01 04:00:00,0.236,0.05


In [ ]:
df1.columns 

Index(['date_time', 'soil_moisture_depth_0.050000', 'depth'], dtype='object')

In [ ]:
df1.describe()

,soil_moisture_depth_0.050000,depth
count,33283.000000,3.364800e+04
mean,0.205819,5.000000e-02
std,0.083589,2.919930e-14
min,0.085000,5.000000e-02
25%,0.139000,5.000000e-02
50%,0.205000,5.000000e-02
75%,0.246000,5.000000e-02
max,0.569000,5.000000e-02


In [ ]:
## split dates and hours :

df1['date_time'][1].split(" ") 

['2017-03-29', '01:00:00']

In [ ]:
## Add hours column 

df1["hour"] = ""


for i in range (len(df1)):
  df1["hour"][i]=df1['date_time'][i].split(" ")[1]
  df1["date_time"][i]=df1['date_time'][i].split(" ")[0]
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'\nfor i in range (8522):\n  df2["hour"][i]=df2[\'date_time\'][i].split(" ")[1]\n  df2["date_time"][i]=df2[\'date_time\'][i].split(" ")[0]\n'

In [ ]:
df1.head()

,date_time,soil_moisture_depth_0.050000,depth,hour
0,2017-03-01,0.238,0.05,00:00:00
1,2017-03-01,0.238,0.05,01:00:00
2,2017-03-01,0.237,0.05,02:00:00
3,2017-03-01,0.236,0.05,03:00:00
4,2017-03-01,0.236,0.05,04:00:00


In [ ]:
Rdataset = pd.DataFrame(columns=['date_time', 'soil_moisture_depth_0.05','Image_name'])

In [ ]:
## Selection of the max value on the day for each image :

L_max = []
for e in df1['date_time'].unique() : 
  L_mid = list(df1[df1['date_time']==e]['soil_moisture_depth_0.050000'])
  L_max.append(max(L_mid))
L_date = df1['date_time'].unique()

In [ ]:
Rdataset['date_time'] = L_date
Rdataset['soil_moisture_depth_0.05'] = L_max
Rdataset.describe()

,soil_moisture_depth_0.05
count,1402.000000
mean,0.211754
std,0.085586
min,0.088000
25%,0.144000
50%,0.210000
75%,0.250750
max,0.569000


In [ ]:
Rdataset.head()

,date_time,soil_moisture_depth_0.05,Image_name
0,2017-03-01,0.240,NaN
1,2017-03-02,0.239,NaN
2,2017-03-03,0.235,NaN
3,2017-03-04,0.230,NaN
4,2017-03-05,0.234,NaN


In [ ]:
import plotly.figure_factory as ff

np.random.seed(1)

x = L_max
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

### As you may notice during the downloading process, we have for the same date 3 different extentions for the same image (QF.tif, UL.tif, TL.tif), but the problem it's not the case for all the images. Soo we. have to pick the the extentions that appear for almost all the images, because with don't want to train our model with different inputs.
### P.S: it might not be the case for you, you can only 1 extention or maybe 2, but just make attention to it.

In [ ]:

directory = '/content/drive/MyDrive/Senti_Images-2017to2020_newArea'
a, b, c = 0, 0, 0
for filename in os.listdir(directory):
    if filename.endswith("TL.tif"):
      a+=1
    if filename.endswith("UL.tif"):
      b+=1
    if filename.endswith("QF.tif"):
      c+=1

In [ ]:
a,b,c

(159, 169, 156)

In [ ]:
## Get a list of the dates from image name :

data_list =[]
Vals = []

for filename in os.listdir(directory):
    if filename.endswith(".tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]
      data_list.append(im_date)

In [ ]:
## Check unique dates:

myset = set(data_list)

In [ ]:
len(data_list)

484

In [ ]:
len(myset)

180

In [ ]:
## Selection of the only useful days and add images names :

directory = '/content/drive/MyDrive/Senti_Images-2017to2020_newArea'
data_list =[]
Vals = []

for filename in os.listdir(directory):
    if filename.endswith("UL.tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]

      if im_date in Rdataset["date_time"].values : 
          index_date= np.where(Rdataset["date_time"]==im_date )
          rdata = Rdataset.iloc[index_date]
          data_list.append(rdata)
          dataset = pd.concat(data_list)
          Vals.append(filename)


dataset['Image_name']= Vals
dataset.head()
      

,date_time,soil_moisture_depth_0.05,Image_name
42,2017-04-12,0.168,20170412T110621_20170412T111708_T30TUL.tif
62,2017-05-02,0.124,20170502T110621_20170502T110937_T30TUL.tif
82,2017-05-22,0.225,20170522T110621_20170522T110912_T30TUL.tif
92,2017-06-01,0.167,20170601T110651_20170601T111225_T30TUL.tif
102,2017-06-11,0.141,20170611T110621_20170611T111012_T30TUL.tif


In [ ]:
## Save dataset :

dataset.to_csv ('Dataset_2017to2020_newArea.csv', header=True, index=False)